<a href="https://colab.research.google.com/github/edwinsalazar/genAIwithAWS/blob/main/Model_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Step 3: Model Fine-tuning
In this notebook, you'll fine-tune the Meta Llama 2 7B large language model, deploy the fine-tuned model, and test it's text generation and domain knowledge capabilities.

Fine-tuning refers to the process of taking a pre-trained language model and retraining it for a different but related task using specific data. This approach is also known as transfer learning, which involves transferring the knowledge learned from one task to another. Large language models (LLMs) like Llama 2 7B are trained on massive amounts of unlabeled data and can be fine-tuned on domain domain datasets, making the model perform better on that specific domain.

Input: A train and an optional validation directory. Each directory contains a CSV/JSON/TXT file.
For CSV/JSON files, the train or validation data is used from the column called 'text' or the first column if no column called 'text' is found.
The number of files under train and validation should equal to one.

- **You'll choose your dataset below based on the domain you've chosen**

Output: A trained model that can be deployed for inference.\
After you've fine-tuned the model, you'll evaluate it with the same input you used in project step 2: model evaluation.

---

#### Set up

---
Install and import the necessary packages. Restart the kernel after executing the cell below.

---

In [26]:
!pip install --upgrade sagemaker datasets
!pip install awscli --quiet
!aws configure
!aws configure get region

AWS Access Key ID [****************B5MN]: 
AWS Secret Access Key [****************yta9]: 
Default region name [us-east-1]: 
Default output format [None]: 
us-east-1


Select the model to fine-tune

In [3]:
model_id, model_version = "meta-textgeneration-llama-2-7b", "2.*"

In the cell below, choose the training dataset text for the domain you've chosen and update the code in the cell below:  

To create a finance domain expert model:

- `"training": f"s3://genaiwithawsproject2024/training-datasets/finance"`

To create a medical domain expert model:

- `"training": f"s3://genaiwithawsproject2024/training-datasets/medical"`

To create an IT domain expert model:

- `"training": f"s3://genaiwithawsproject2024/training-datasets/it"`

In [27]:
from sagemaker.jumpstart.estimator import JumpStartEstimator
import boto3
!aws configure

role_arn = "arn:aws:iam::317957367373:role/SageMakerExecutionRole"

model_id = "meta-textgeneration-llama-2-7b"

estimator = JumpStartEstimator(
    model_id=model_id,
    environment={"accept_eula": "true"},
    instance_type="ml.g5.8xlarge",
    role=role_arn
)

estimator.set_hyperparameters(instruction_tuned="False", epoch="5")

training_data_s3_path = "s3://sagemaker-us-east-1-317957367373/financialDataset.txt"

estimator.fit({"training": training_data_s3_path})

AWS Access Key ID [****************B5MN]: 
AWS Secret Access Key [****************yta9]: 
Default region name [us-east-1]: 
Default output format [None]: 
2024-10-08 21:01:01 Starting - Starting the training job
2024-10-08 21:01:01 Pending - Training job waiting for capacity......
2024-10-08 21:01:38 Pending - Preparing the instances for training...
2024-10-08 21:02:19 Downloading - Downloading input data...........................
2024-10-08 21:07:28 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-10-08 21:07:45,320 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-10-08 21:07:45,339 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-10-08 21:07:45,349 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-10-08 21:07:45,35

#### Deploy the fine-tuned model
---
Next, we deploy the domain fine-tuned model. We will compare the performance of the fine-tuned and pre-trained model.

---

In [32]:
#from sagemaker.serializers import JSONSerializer
#from sagemaker.deserializers import JSONDeserializer

finetuned_predictor = estimator.deploy(

#predictor = estimator.deploy(
     initial_instance_count=1,
     instance_type="ml.g5.4xlarge",
#    serializer=JSONSerializer(),
#    deserializer=JSONDeserializer(),
)

-----------!

#### Evaluate the pre-trained and fine-tuned model
---
Next, we use the same input from the model evaluation step to evaluate the performance of the fine-tuned model and compare it with the base pre-trained model.

---

Create a function to print the response from the model

In [40]:
def print_response(payload, response):
    print(payload["inputs"])
    print(f"> {response}")
    print("\n==================================\n")

Now we can run the same prompts on the fine-tuned model to evaluate it's domain knowledge.   

**Replace "inputs"** in the next cell with the input to send the model based on the domain you've chosen.

**For financial domain:**

  "inputs": "Replace with sentence below from text"  
- "The  investment  tests  performed  indicate"
- "the  relative  volume  for  the  long  out  of  the  money  options, indicates"
- "The  results  for  the  short  in  the  money  options"
- "The  results  are  encouraging  for  aggressive  investors"

**For medical domain:**

  "inputs": "Replace with sentence below from text"
- "Myeloid neoplasms and acute leukemias derive from"
- "Genomic characterization is essential for"
- "Certain germline disorders may be associated with"
- "In contrast to targeted approaches, genome-wide sequencing"

**For IT domain:**

  "inputs": "Replace with sentence below from text"
- "Traditional approaches to data management such as"
- "A second important aspect of ubiquitous computing environments is"
- "because ubiquitous computing is intended to"
- "outline the key aspects of ubiquitous computing from a data management perspective."

In [41]:
payload = {
    "inputs": "the relative volume for the long out of the money options, indicates",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = finetuned_predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

the relative volume for the long out of the money options, indicates
> [{'generated_text': ' that the market maker is trying to keep the price in check.\nThe market maker will try to make sure that the stock price does not rise too much, because if it does, then the long out of the money options will lose value.\nThe market maker will then try to make sure that the'}]




In [42]:
payload = {
    "inputs": "The results are encouraging for aggressive investors",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = finetuned_predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

The results are encouraging for aggressive investors
> [{'generated_text': ", but there are a number of risks to be aware of.\nThe S&P 500 (SNPINDEX:^GSPC) has been on a tear lately, with the index rising by 13% in 2019 alone. The market's strong"}]




In [43]:
payload = {
    "inputs": "The results for the short in the money options",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = finetuned_predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

The results for the short in the money options
> [{'generated_text': ' are also shown in the chart below.\nThe results for the short in the money options are also shown in the chart below.\nThe results for the short in the money options are also shown in the chart below.\nThe results for the short in the money options are also shown in the chart below.\nThe'}]




In [44]:
payload = {
    "inputs": "The investment tests performed indicate",
    "parameters": {
        "max_new_tokens": 64,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}
try:
    response = finetuned_predictor.predict(payload, custom_attributes="accept_eula=true")
    print_response(payload, response)
except Exception as e:
    print(e)

The investment tests performed indicate
> [{'generated_text': ' that the proposed hybrid system can be a good alternative to the traditional grid-connected solar systems.\nInvestment analysis and payback period calculation were used to determine the economic viability of the proposed system. The payback period of the proposed system is 1.3 years, which is much shorter than the'}]




Do the outputs from the fine-tuned model provide domain-specific insightful and relevant content? You can continue experimenting with the inputs of the model to test it's domain knowledge.

**Use the output from this notebook to fill out the "model fine-tuning" section of the project documentation report**

**After you've filled out the report, run the cells below to delete the model deployment**

`IF YOU FAIL TO RUN THE CELLS BELOW YOU WILL RUN OUT OF BUDGET TO COMPLETE THE PROJECT`

In [45]:
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()